In [ ]:
# This notebook should run through the entire process of the experiment:
# preprocessing the data, processing it, analyzing it, and turning it into graphs

# For debugging; remove when no longer needed
%load_ext autoreload
%autoreload 2

# Any global imports we need
from collections import Counter, defaultdict
from pathlib import Path
import pickle
import bz2

from tqdm.notebook import tqdm # Shiny progress bars

In [ ]:
# First of all, let's do the German/English experiment
import celex

In [ ]:
# First we set up the data file (data/process.py)
import data.process
from csv import QUOTE_NONE
# The first step is to convert the CELEX data to a format we can work with
# This expects it to be in CSV format to start with (like the data provided by Dr Oh)
# If you have raw HTML data exported from WebCelex, see data/preprocess.sh
raw_file = 'data/deu_modified.csv' # The data provided by Oh
data_file = 'data/german_TMP.pickle.bz2' # What we'll be exporting to
parameters = {'delimiter':'\t', 'escapechar':'\x1b', 'strict':True, 'quoting':QUOTE_NONE} # The parameters of Oh's data
# (See data/process.py for what delimiters should be used for raw CELEX data)
counter = 'Word Mann'
data.process.do_processing(parameters, raw_file, data_file, counter)
print('Data saved to', data_file)

In [ ]:
# Now we do the actual analysis (celex.py, analysis.py)
parameters = {'stress':True, 'freq':'Word Mann', 'phon':'DISC', 'divider':' '}
# For English, do {'stress':True, 'freq':'CobW', 'phon':'DISC'}
# See celex.py for explanation
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)

In [ ]:
# Now we have an analysis of German that we can use in various ways!
# Here are the most important ones
e1, e2 = analysis.do_things()
print('Shannon entropy:', e1, 'Conditional entropy:', e2)
analysis.corpus = Counter(dict(Counter(analysis.corpus).most_common(20000))) # Cut down to 20k most common
analysis.count_unigrams()
print('Syllable types in 20k most common:', len(analysis.unigrams))

# Generate a new analysis object
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)
# Calculate the e2 value (conditional entropy) at different sizes of corpus
# On a log scale with 200 points, repeated 5 times
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, bootstrap=False,
                                save='math/german_log_TMP.pickle.bz2')
print('German base finished')

# And one more new analysis object
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)
# Same as above but only up to a limit of 2m
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, top=2_000_000, bootstrap=False, cut_top=True,
                                save='math/german_log_cut2_TMP.pickle.bz2')
print('German cut2 finished')

In [ ]:
# Now let's set up English in the same way
# Once again, start with CSV; if you don't have CSV, use data/preprocess.sh
raw_file = 'data/eng.csv' # Data scraped from WebCelex
data_file = 'data/english_TMP.pickle.bz2' # What we'll be exporting to
parameters = {'delimiter':'\\', 'escapechar':'\x1b', 'strict':True, 'quoting':QUOTE_NONE} # Different delimiter here
counter = 'CobW'
data.process.do_processing(parameters, raw_file, data_file, counter)
print('Data saved to', data_file)

In [ ]:
# And analyze it
parameters = {'stress':True, 'freq':'CobW', 'phon':'DISC'}
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)

In [ ]:
# Then, save some results for English
# Now we have an analysis of German that we can use in various ways!
# Here are the most important ones
e1, e2 = analysis.do_things()
print('Shannon entropy:', e1, 'Conditional entropy:', e2)
analysis.corpus = Counter(dict(Counter(analysis.corpus).most_common(20000))) # Cut down to 20k most common
analysis.count_unigrams()
print('Syllable types in 20k most common:', len(analysis.unigrams))

# Generate a new analysis object
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)
# Calculate the e2 value (conditional entropy) at different sizes of corpus
# On a log scale with 200 points, repeated 5 times
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, bootstrap=False,
                                save='math/english_log_TMP.pickle.bz2')
print('English base finished')

# Another more new analysis object
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)
# Same as above but only up to a limit of 2m
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, top=2_000_000, bootstrap=False, cut_top=True,
                                save='math/english_log_cut2_TMP.pickle.bz2')
print('English cut2 finished')

# Then for English, we do one test we didn't do for German: bootstrapping to higher size
analysis = celex.CelexAnalysis(**parameters, log=False)
analysis.load_corpus(data_file)
top = analysis.tokens * 10 # Go one order of magnitude higher than we have
analysis.calculate_reduced_e2(n=2, logscale=True, npts=200, bootstrap=True, top=top,
                              save='math/english_bootstrap_TMP.pickle.bz2')
print('English bootstrap finished')

In [ ]:
# Now it's time for the most important part: the Latin analysis!
# This assumes you have the PHI plaintext data stored in the appropriate place
# For me, that's ~/cltk_data/latin/text/phi5
# I'm not sure if I can legally distribute it, but you should be able to find it online without too much difficulty

# You also need my custom version of cltk installed
# Put the path to that here
import sys
sys.path.insert(1, '../CLTK/cltk/')
import cltk

# WARNING: For some reason, this sometimes (inconsistently!) throws errors with recent versions of NLTK.
# If you get errors, roll back to NLTK version 3.5 (pip3 install nltk==3.5) and that should fix it.
# I suspect this has to do with breaking changes made to NLTK, which CLTK was updated to support...
# ...but my fork doesn't have those fixes, and I'm hesitant to mess with it much at this point.
# So rolling back NLTK is the safer choice.

import data.latin.corpus as cps
JUSTINIAN = 'LAT2806' # Justinian's author code

solo = Path('./data/latin/auth_solo_TMP') # The directory where authors' solo corpora are stored
solo.mkdir(exist_ok=True) # Create it if it doesn't already exist
print('Done')

In [ ]:
# We first generate an overview of authors' token counts, to ensure the corpus is working
# If this doesn't work, your PHI5 or CLTK installation is broken
# (This is designed to test the macronizer and syllabifier as well, so it takes a while)
data = cps.PHI5Corpus().get_author_data()
with bz2.open('authors_TMP.pickle.bz2', 'w') as f:
    pickle.dump(data, f)
with open('authors_TMP.tsv', 'w') as f:
    for (name, id), words in sorted(data.items()):
        f.write(f'{name}\t{id}\t{words}\n')
for (name, id), words in sorted(data.items(), key=lambda x:x[1], reverse=True)[:5]:
    print(name, id, words) # This will show you if it's working or not

In [ ]:
# We need to generate two versions, one with Justinian (_complete) and one without
# But since we're rewriting the code for this notebook, why not do it in a more efficient way?
# Let's only generate the smaller one, and then add Justinian to it
# (Why generate this at all, instead of just going over each author once, and then stitching them together?)
# (Because this gives us some extra redundancy against errors, which was very important early on in the project!)
CORPUS_SMALL = './data/latin/phi5_TMP.pickle.bz2'
cps.PHI5Corpus().process_and_save(CORPUS_SMALL, authorial=True, shuffle=True, exclude=(JUSTINIAN,), overwrite=False)
# Make overwrite=True to replace a previous run if necessary

In [ ]:
CORPUS_BIG = './data/latin/phi5_complete_TMP.pickle.bz2'
tmp = solo/(JUSTINIAN+'.pickle.bz2') # We're doing this before we deal with the rest of the solo folder, but why duplicate effort?
cps.PHI5Corpus().process_and_save(tmp, authorial=True, shuffle=True, include=(JUSTINIAN,), overwrite=False)
with bz2.open(CORPUS_SMALL, 'r') as f:
    small = Counter(pickle.load(f))
with bz2.open(tmp, 'r') as f:
    just = Counter(pickle.load(f))
new = small + just
with bz2.open(CORPUS_BIG, 'w') as f:
    pickle.dump(dict(new), f)

In [ ]:
# Then we generate corpora for each "important" author individually, plus a MISC one for all the others
# Justinian is specifically excluded from all of these!
c = cps.PHI5Corpus()
IMPORTANT_PLUS_JUSTINIAN = cps.IMPORTANT_AUTHORS | {JUSTINIAN} # Exclude Justinian from the MISC category
c.process_and_save(solo/'MISC.pickle.bz2', exclude=IMPORTANT_PLUS_JUSTINIAN,
                   overwrite=False, shuffle=True, authorial=True) # Change overwrite to true to replace an existing file
for auth in tqdm(cps.IMPORTANT_AUTHORS):
    c.process_and_save(solo/f'{auth}.pickle.bz2', authorial=True, include=(auth,), overwrite=False, shuffle=True)
    # Again, change overwrite to true if you want to replace an existing file
    # It's left false by default so that, if this cell gets interrupted, you can skip the ones that were already finished
# Justinian was handled in a previous cell, but if you need to:
# cps.process_and_save(solo/(JUSTINIAN+'.pickle.bz2'), authorial=True, include=(JUSTINIAN,), overwrite=False, shuffle=True)

In [ ]:
# And subtract each of them from the complete collection
def subtract_authors(complete=False):
    result = Path('./data/latin/') / ('auth_complete_TMP' if complete else 'auth_TMP')
    result.mkdir(exist_ok=True)
    #c = cps.PHI5Corpus() # We don't actually need a corpus object here, we're just working with Counters
    with bz2.open(CORPUS_BIG if complete else CORPUS_SMALL, 'r') as f: # Choose the appropriate full corpus
        full = Counter(pickle.load(f))
    for auth in tqdm(list(solo.glob('*.pickle.bz2'))):
        if auth.name.startswith(JUSTINIAN) and not complete:
            print('(Skipping Justinian)')
            continue
        with bz2.open(auth, 'r') as f:
            d = Counter(pickle.load(f))
        new = full - d
        if sum(new.values()) != sum(full.values()) - sum(d.values()): # Sanity check
            raise ValueError(auth, sum(new.values()), sum(full.values()), sum(d.values()))
        with bz2.open(result / auth.name, 'w') as f:
            pickle.dump(dict(new), f)

In [ ]:
# Run it with Justinian
subtract_authors(complete=True)

In [ ]:
# Run it without Justinian
subtract_authors(complete=False)

In [ ]:
# And now that we have all this data, we can also calculate accurate type and token counts for each author
with bz2.open(CORPUS_BIG, 'r') as f:
    d_tot = pickle.load(f)
ctype, ctoken = len(d_tot), sum(d_tot.values())
print('Total (big)', ctype, ctoken)
with bz2.open(CORPUS_SMALL, 'r') as f:
    d_tot = pickle.load(f)
ctype, ctoken = len(d_tot), sum(d_tot.values())
print('Total (small)', ctype, ctoken)
for auth in solo.glob('*.pickle.bz2'):
    with bz2.open(auth, 'r') as f:
        d = pickle.load(f)
    ttype, ttoken = len(d), sum(d.values())
    print(auth.stem, ttype, ttoken)
    if auth.name.startswith(JUSTINIAN):
        print('\t(That\'s Justinian; skipped)')
    else:
        ctype -= ttype; ctoken -= ttoken
print('Unaccounted for', ctype if ctype > 0 else 0, ctoken) # We expect ctype to be negative because many authors use the same types
# But ctoken should definitely be zero, and if it's not, then something's gone wrong

In [ ]:
# Now that we have our various Latin corpora produced, we can analyze them the same way we did English and German
# See analyze.py for details on this
import analyze

In [ ]:
# Calculate raw entropy, first of all
analysis = analyze.Analysis()
analysis.load_corpus(CORPUS_SMALL)
e1, e2 = analysis.do_things()
print('Shannon entropy', e1, 'Conditional entropy', e2)

In [ ]:
# Now cut it down randomly for extrapolation purposes
# This takes less time than for English or German because the corpus is smaller
analysis = analyze.Analysis(log=False)
analysis.load_corpus(CORPUS_SMALL)
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, bootstrap=False,
                                save='math/latin_log_TMP.pickle.bz2')
print('Done') # Have to do something else afterward to avoid Jupyter printing all the raw output from the analysis

In [ ]:
# And do the same for the complete corpus, including Justinian (in order to compare the two)
analysis = analyze.Analysis(log=False)
analysis.load_corpus(CORPUS_BIG)
analysis.calculate_reduced_e2(logscale=True, npts=200, n=5, bootstrap=False,
                                save='math/latin_log_complete_TMP.pickle.bz2')
print('Done')

In [ ]:
out = Path('./math/latin_auth_TMP')
out.mkdir(exist_ok=True)
inp = Path('./data/latin/auth_TMP')
for auth in tqdm(list(inp.glob('*.pickle.bz2'))):
    if auth.name.startswith(JUSTINIAN) or auth.name.startswith('MISC'):
        print('Skipping', auth.name)
        continue
    path = out / auth.name
    if path.exists() and True: # Change True to False to overwrite existing ones
        print(auth.name, 'already exists')
        continue
    analysis = analyze.Analysis(log=False)
    analysis.load_corpus(auth)
    analysis.calculate_reduced_e2(logscale=True, npts=200, n=1, bootstrap=False, # Only n=1 for these
                                  save=path)
# Strictly speaking we should then do the same again, with the _complete corpus
# But for the sake of time I'm not doing that here; it works exactly the same as the above
# Just change out and inp to latin_auth_complete_TMP and latin/auth_complete_TMP

In [ ]:
# Now we've got all the data files we need, stored safely in ./math
# The next step is to plot 'em!
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

import plots
# See plots.py for the exact details of how the figures were generated
# Warning: depending on your version of Numpy, the curve-fitting may produce spurious warnings!
# Numpy warns whenever you take a fractional power of a negative number, even if the result is real
# Everything works fine, though, and you can safely ignore these

In [ ]:
# Curve-fitting for German

plt.rcParams.update({'font.size': 12})

d = plots.Dataset('math/german_log_TMP.pickle.bz2')
d.fit_curve()
d.mark_curve(xmax = max(d.xs)*10)
d.draw_data('b.')
plt.plot(d.xs, d.ys, '.', color='#70c070', label='Data') # Green for German
d.draw_curve('-k')
plt.plot(d.pxs, d.pys, '-k', alpha=0)
d.draw_asymptote('--', 'g', False)
plt.axhline(y=d.popt[0], linestyle='--', color='r', alpha=0)
plt.xlabel('Corpus Size (tokens)')
plt.ylabel('Information Density (bits/syl)')
print(d.popt)
print(max(d.ys))

newticks = [6.4]
plt.yticks(list(plt.yticks()[0]) + newticks) # Add an extra tick to the y-axis
plt.legend()

d.show()

In [ ]:
# Extrapolating English and German

plt.rcParams.update({'font.size': 12})

d2 = plots.Dataset('math/english_log_cut2_TMP.pickle.bz2')
d2.fit_curve()
d2.mark_curve(xmax = max(d2.xs)*10)
plt.plot(d2.xs, d2.ys, '.', color='#7070ff', label='English')
d2.draw_asymptote('-', 'b', False, label=None)
d2.draw_curve('-k', label=None)
print(d2.popt)
print(max(d2.ys))
d2.draw_asymptote('--', 'r', False, override=6.98057, label=None)

d3 = plots.Dataset('math/german_log_cut2_TMP.pickle.bz2')
d3.fit_curve()
d3.mark_curve(xmax = max(d3.xs)*10)
plt.plot(d3.xs, d3.ys, '.', color='#70c070', label='German')
d3.draw_asymptote('-', 'g', False, label=None)
d3.draw_curve('-k')
print(d3.popt)
print(max(d3.ys))
d3.draw_asymptote('--', 'r', False, override=6.082567690505002, label="Previous Result")
plt.legend()
plt.xlabel('Corpus Size (tokens)')
plt.ylabel('Information Density (bits/syl)')

d3.show()

In [ ]:
# Latin!

plt.rcParams.update({'font.size': 12})
d = plots.Dataset('math/latin_log_TMP.pickle.bz2')
d.fit_curve()
d.mark_curve(xmax = max(d.xs)*10) # *10 for Latin, *100 if you're doing German
plt.plot(d.xs, d.ys, '.', color='#70c4ff', label='Data')
d.draw_curve('-k')
d.draw_asymptote('--', 'b', False)
plt.xlabel('Corpus Size (tokens)')
plt.ylabel('Information Density (bits/syl)')
print(d.popt)
print(max(d.ys))
plt.legend()
d.show()

In [ ]:
# Comparison of authors

plt.rcParams.update({'font.size': 12})
vals = []

d0 = plots.Dataset('math/latin_log_TMP.pickle.bz2') # Use _complete to include the Digesta
d0.fit_curve()

for i, auth in enumerate(Path('math/latin_auth_TMP').glob('*.pickle.bz2')): # Use _complete to include the Digesta
	if '0474' in auth.stem and False: continue # Remove Cicero here if you want
	d = plots.Dataset(auth)
	d.fit_curve()
	d.mark_curve(xmax=max(d0.xs)*10, npts=500)
	# old: random.choice('oDv^s')
	d.draw_data('.'+'bgmycbgmycbgcmyb'[i])
	d.draw_curve('k')
	vals.append(d.popt[0])
	print(f'Without {auth.stem.split(".")[0]}: {d.popt[0]}')

d0.draw_asymptote('-', 'r', False)

plt.xlabel('Corpus Size (tokens)')
plt.ylabel('Information Density (bits/syl)')

print(f'Actual value: {d0.popt[0]}')
print(f'Approximated value: {sum(vals)/len(vals)}')
print(f'Approximated standard error: {np.std(vals, ddof=1)}')

d0.show()

In [ ]:
plt.rcParams.update({'font.size': 12})

d = plots.Dataset('math/latin_log_complete_TMP.pickle.bz2')
d.fit_curve()
d.mark_curve(xmax = max(d.xs)*10)
plt.plot(d.xs, d.ys, '.', color='#ff7070', label='With Digesta')
print(d.popt)

d2 = plots.Dataset('math/latin_log_TMP.pickle.bz2')
d2.fit_curve()
d2.mark_curve(xmax = max(d2.xs)*10)
plt.plot(d2.xs, d2.ys, '.', color='#70c4ff', label='Without Digesta')
print(d2.popt)

d.draw_asymptote('--', 'r', False, label=None)
d2.draw_asymptote('--', 'b', False, label=None)

d.draw_curve('-k', label=None)
d2.draw_curve('-k', label=None)

plt.xlabel('Corpus Size (tokens)')
plt.ylabel('Information Density (bits/syl)')
plt.legend(loc='lower right')
d2.show()